In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [3]:
data = pd.read_csv('/content/Churn_Modelling.csv')
data.head()


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
data = data.drop(['RowNumber', 'CustomerId', 'Surname'], axis =1)

In [5]:
## Encoding categorical variables

Label_encoder_gender = LabelEncoder()
data['Gender'] = Label_encoder_gender.fit_transform(data['Gender'])
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [6]:
## Encode Geography
onehot_encoder_geo = OneHotEncoder(handle_unknown='ignore')
geo_encoded = onehot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded, columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [7]:
data = pd.concat([data.drop('Geography', axis =1), geo_encoded_df],axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [8]:
# Split the data into features and target
X = data.drop('EstimatedSalary', axis = 1)
y = data['EstimatedSalary']

In [9]:
import pickle

In [10]:
## Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.2, random_state= 42)

In [11]:
##Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [12]:
##Save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
  pickle.dump(Label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
  pickle.dump(onehot_encoder_geo,file)

with open('scaler.pkl', 'wb') as file:
  pickle.dump(scaler, file)

In [13]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [14]:
### Build Our ANN Model
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (X_train.shape[1],)), ##Input layer connected to the hidden layer
    Dense(32, activation = 'relu'), #Hidden layer
    Dense(1) ## Output Layer -- DEFAULT IS LINEAR ACTIVATION FUNCTION
]
      )

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
## Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

In [16]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

log_dir = "regressionlogs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq=1)

In [17]:
## Set up earlystopping
early_stopping_callback = EarlyStopping(monitor= 'val_loss', patience= 10, restore_best_weights = True)

In [18]:
## Train the model
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs = 100,
    callbacks= [tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - loss: 99897.7891 - mae: 99897.7891 - val_loss: 98479.9062 - val_mae: 98479.9062
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 99872.8594 - mae: 99872.8594 - val_loss: 96686.6016 - val_mae: 96686.6016
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 96931.6406 - mae: 96931.6406 - val_loss: 92105.0469 - val_mae: 92105.0469
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 92850.7812 - mae: 92850.7812 - val_loss: 84651.3984 - val_mae: 84651.3984
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 84419.5547 - mae: 84419.5547 - val_loss: 75287.4219 - val_mae: 75287.4219
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 74271.9922 - mae: 74271.9922 - val_loss: 66089.7031 - val_mae: 66089.7031
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 65311.8984 - mae: 65311.8984 - val_loss: 58710.9688 - val_mae: 58710.9688
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - 

In [19]:
model.save('regression_model.h5')

In [20]:
## Load Tensorboard Extension
%load_ext tensorboard

In [21]:
%tensorboard --log_dir regressionlogs/fit

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:292: SyntaxWarning: invalid escape sequence '\s'
  "[`\000-\040\177-\240\s]+",
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:339: SyntaxWarning: invalid escape sequence '\s'
  style = re.compile('url\s*\(\s*[^\s)]+?\s*\)\s*').sub(' ', style)
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:354: SyntaxWarning: invalid escape sequence '\s'
  if not re.match("^\s*([-\w]+\s*:[^:;]*(;\s*|$))*$", style):
/usr/local/lib/python3.12/dist-packages/tensorboard/_vendor/bleach/sanitizer.py:358: SyntaxWarning: invalid escape sequence '\w'
  for prop, value in re.findall('([-\w]+)\s*:\s*([^:;]*)', style):
2025-11-07 11:59:34.810296: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one ha

In [22]:
## Evaluate model on Test data
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test MAE : {test_mae}')

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 51117.1953 - mae: 51117.1953
Test MAE : 50365.1640625
